In [ ]:
# Classificação de Sentimentos - Tensor Flow

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
!tree


.
├── Classificador de Sentimentos.ipynb
├── Datasets
│   ├── NoThemeTweets.csv
│   ├── testdatasets
│   │   ├── Test3classes.csv
│   │   ├── Test.csv
│   │   └── TestTema.csv
│   ├── TestDatasets
│   │   ├── Test3classes.csv
│   │   ├── Test.csv
│   │   └── TestTema.csv
│   ├── trainingdatasets
│   │   ├── Train100.csv
│   │   ├── Train200.csv
│   │   ├── Train300.csv
│   │   ├── Train3Classes.csv
│   │   ├── Train400.csv
│   │   ├── Train500.csv
│   │   ├── Train50.csv
│   │   └── TrainTema.csv
│   ├── TrainingDatasets
│   │   ├── Train100.csv
│   │   ├── Train200.csv
│   │   ├── Train300.csv
│   │   ├── Train3Classes.csv
│   │   ├── Train400.csv
│   │   ├── Train500.csv
│   │   ├── Train50.csv
│   │   └── TrainTema.csv
│   ├── TweetsNeutralHash.csv
│   ├── TweetsNeutralNews.csv
│   └── TweetsWithTheme.csv
└── README.md

5 directories, 28 files


In [11]:

df_train = pd.read_csv("Datasets/trainingdatasets/Train500.csv",encoding='utf-8', sep=';',engine='python')
df_train.head()
stopword = pd.read_table("stopwords.txt")

In [12]:
df_train.drop(['id','tweet_date','query_used'],axis=1,inplace=True)
df_train.head()

,tweet_text,sentiment
0,@Laranjito76 A pessoa certa para isso seria o ...,1
1,"@behin_d_curtain Para mim, é precisamente o co...",1
2,Vou fazer um video hoje... estou pensando em f...,1
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1
4,Valoriza o coração do menininho que vc tem. El...,1


In [22]:
stoplist = []
sentences = []
for i in range(len(stopword)):
    stoplist.append(stopword['stopwords'][i])
    
for i in range(len(df_train['tweet_text'])):
    sentence = df_train['tweet_text'][i]
    for word in stoplist:
        newword = " "+word+" "
        sentence = sentence.replace(newword," ")
        sentence = sentence.replace("  "," ")
    sentences.append(sentence)
    
    

agora vamos remover os emojis

In [29]:
import re
def emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


In [28]:
sentences = map(emoji,s)

'aaaaaaaa amei tanto essas polaroids, nem sei expressar o quanto eu to apaixonada de vdd espero que outras pessoas consigam ganhar também :'